# pandas

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('music_streaming.csv')
df.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


- handling missing values

In [3]:
print(df.isnull().sum())

Artist Name              0
Track Name               0
Popularity             394
danceability             0
energy                   0
key                   1743
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      3587
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Genre                    0
dtype: int64


In [4]:
popularttiy_median = df['Popularity'].median()
df['Popularity'].fillna(popularttiy_median, inplace=True)

key_mode = df['key'].mode().iloc[0]
df['key'].fillna(key_mode, inplace=True)

instrument_mode = df['instrumentalness'].mode().iloc[0]
df['instrumentalness'].fillna(instrument_mode, inplace=True)

In [5]:
print(df.isnull().sum())

Artist Name           0
Track Name            0
Popularity            0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_in min/ms    0
time_signature        0
Genre                 0
dtype: int64


- Drop duplicates

In [6]:
df.drop_duplicates(inplace=True)

- Add new column

In [7]:
df['duration_in_min'] = df['duration_in min/ms'] / 60000
df['duration_in_min']

0         3.909933
1         4.195550
2         1.827783
3         2.899467
4         3.832667
           ...    
15512    10.750000
15513     3.760450
15514     0.000081
15515     5.013667
15516     4.246750
Name: duration_in_min, Length: 15517, dtype: float64

# SparkSQL

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.format("csv").option("header", "true").load("after_cleaning.csv")
df.createOrReplaceTempView("songs")


- a) Perform any necessary data cleaning & engineering that renders your data useable (i.e. handling
missing values, duplicates, classification, transformation...etc.)

- b) Remove any songs that exceed 5 minutes


In [9]:
# This query filters out songs that exceed a duration of 5 minutes, by considering songs within this duration limit
after_removing = spark.sql("SELECT * FROM songs WHERE duration_in_min <= 5")

- c) Display songs by J. Cole, Novo Amor and Anson Seabra

Backticks (`) are used to enclose column names that have special characters, spaces, or reserved keywords.

Single quotes (') are used to enclose string values or literals in SQL queries.

In [10]:
# displays the songs by the artists "J. Cole", "Novo Amor", and "Anson Seabra" by selecting name of them
# use of the " IN " operator simplifies the query by allowing multiple values to be checked against the "Artist Name" 
spark.sql("SELECT * FROM songs WHERE `Artist Name` IN ('J. Cole', 'Novo Amor', 'Anson Seabra')").show()

+------------+--------------------+----------+------------+------+----+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+--------------+-----+------------------+
| Artist Name|          Track Name|Popularity|danceability|energy| key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_in min/ms|time_signature|Genre|   duration_in_min|
+------------+--------------------+----------+------------+------+----+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+--------------+-----+------------------+
|     J. Cole|p u n c h i n ‚Äò...|      81.0|       0.769|  0.74|11.0|  -6.579|   0|      0.378|       0.197|        0.000109|   0.171|  0.692| 94.104|          112579.0|             4|    5|1.8763166666666666|
|     J. Cole|p r i d e . i s ....|      85.0|       0.861| 0.656| 4.0|  -7.867|   0|      0.425|       0.104|        0.000109|  0.0986|  0.331| 88.967|

- d) How many songs are included in every category?


In [11]:
# this query provides the distribution of songs across different genres by counting the occurrences of each genre
# the COUNT(*) function is used to count the number of rows 
song_per_category = spark.sql("SELECT Genre, COUNT(*) AS Count FROM songs GROUP BY Genre")
song_per_category.show()

+-------+-----+
|  Genre|Count|
+-------+-----+
|      7|  464|
|117.017|    1|
|      3|  371|
|      8| 1704|
|      0|  586|
|      5| 1210|
|      6| 2263|
|      9| 1828|
|      1| 1268|
|     10| 4264|
|      4|  376|
|      2| 1182|
+-------+-----+



- e) Which artists dominated the charts?


In [12]:
# query identifies the artists who have the highest number of songs in the dataset by counting the occurrences of each artist's name
# ordering the results in descending order to provides a ranking of artists in terms of the number of songs they have
dominant_artists = spark.sql("SELECT `Artist Name`, COUNT(*) AS Count FROM songs group by `Artist Name` ORDER BY count desc")
dominant_artists.show()

+--------------------+-----+
|         Artist Name|Count|
+--------------------+-----+
|     Backstreet Boys|   66|
|            Westlife|   54|
|      Britney Spears|   52|
|  The Rolling Stones|   36|
|                  U2|   29|
|           Metallica|   27|
|     Lata Mangeshkar|   25|
|               AC/DC|   23|
|         The Beatles|   23|
|       Fleetwood Mac|   22|
|        Led Zeppelin|   22|
|             Nirvana|   22|
|      The Black Keys|   22|
|       Mohammed Rafi|   21|
|            Coldplay|   20|
|Creedence Clearwa...|   20|
|       Kishore Kumar|   20|
|The Smashing Pump...|   18|
|           Pearl Jam|   18|
|           Aerosmith|   18|
+--------------------+-----+
only showing top 20 rows



- f) What song would be considered for the “Billboard Top 10 song of the Year”?
(mention their artists as well)


In [13]:
# query retrieves the top 10 most popular songs based on the popularity column in the dataset
top_ten = spark.sql("SELECT `Track Name`, `Artist Name` FROM songs order by Popularity desc limit 10")
top_ten.show()

+--------------------+--------------------+
|          Track Name|         Artist Name|
+--------------------+--------------------+
|            good 4 u|      Olivia Rodrigo|
|Kiss Me More (fea...|            Doja Cat|
|Astronaut In The ...|         Masked Wolf|
|STAY (with Justin...|The Kid LAROI, Ju...|
|          Bad Habits|          Ed Sheeran|
|STAY (with Justin...|The Kid LAROI, Ju...|
|              Butter|                 BTS|
|             RAPSTAR|              Polo G|
|             RAPSTAR|              Polo G|
|                Fiel|     Los Legendarios|
+--------------------+--------------------+



- g) Recommend at least 5 songs that can be played at a party

In [14]:
# Rock, Indie, Alt, Pop, Metal, HipHop, Alt_Music, Blues, Acoustic/Folk, Instrumental, Country
#    1,     2,   3,   4,     5,      6,         7,     8,             9,           10,      11

spark.sql("SELECT `Track Name`, `Artist Name` FROM songs  ORDER BY popularity DESC LIMIT 5 ").show()



+--------------------+--------------------+
|          Track Name|         Artist Name|
+--------------------+--------------------+
|            good 4 u|      Olivia Rodrigo|
|Kiss Me More (fea...|            Doja Cat|
|STAY (with Justin...|The Kid LAROI, Ju...|
|          Bad Habits|          Ed Sheeran|
|Astronaut In The ...|         Masked Wolf|
+--------------------+--------------------+



# SparkDataframes

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Spotify").getOrCreate()

df = spark.read.format("csv").option("header", "true").load("after_cleaning.csv")

- a) Perform any necessary data cleaning & engineering that renders your data useable (i.e. handling
missing values, duplicates, classification, transformation...etc.)

- b) Remove any songss that exceed 5 minutes

- c) Display songss by J. Cole, Novo Amor and Anson Seabra

- d) How many songss are included in every category?

- e) Which artists dominated the charts?

- f) What songss would be considered for the “Billboard Top 10 songss of the Year”?
(mention their artists as well)

- g) Recommend at least 5 songss that can be played at a party